In [1]:
!pip install huggingface_hub

#LLaVA
!pip install transformers==4.37.2
!echo Y | pip uninstall flash-attn
!pip install -e ".[train]"

!pip install -q accelerate peft bitsandbytes
#restart the session once done with installiation of "sentencepiece" and "protobuf"
!pip install sentencepiece
!pip install protobuf
!pip install torch==2.1.2 torchvision==0.16.2 triton==2.1.0 accelerate==0.26.1 deepspeed==0.13.1 pynvml==11.5.0 --upgrade
!pip install flash-attn --no-build-isolation --no-cache-dir

##LLaVA-Med
!pip uninstall torch torchvision -y
!pip install torch==2.0.0+cu117 torchvision==0.15.1+cu117 torchaudio==2.0.1 --index-url https://download.pytorch.org/whl/cu117
!pip install openai==0.27.8
!pip uninstall transformers -y
!pip install git+https://github.com/huggingface/transformers@cae78c46
!pip install -e .


!pip install gradio

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.4/129.4 kB 6.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.4/8.4 MB 89.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.6/3.6 MB 71.8 MB/s eta 0:00:00
  Attempting uninstall: tokenizers
    Found existing installation: tokenizers 0.19.1
    Uninstalling tokenizers-0.19.1:
      Successfully uninstalled tokenizers-0.19.1
  Attempting uninstall: transformers
    Found existing installation: transformers 4.44.2
    Uninstalling transformers-4.44.2:
      Successfully uninstalled transformers-4.44.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
sentence-transformers 3.2.1 requires transformers<5.0.0,>=4.41.0, but you have transformers 4.37.2 which is incompatible.
Obtaining file:///content
ERROR: file:///content does not appear to be a Python project: neither 'setup.py' nor 'pypr

In [3]:
%cd /content/drive/MyDrive/Colab Notebooks/Medical_Project/My_new_LLaVA/LLaVA

/content/drive/MyDrive/Colab Notebooks/Medical_Project/My_new_LLaVA/LLaVA


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
import json
import os
from transformers import AutoProcessor, AutoModelForVision2Seq, pipeline
import torch
from PIL import Image
import gradio as gr
import subprocess
from llava.model.builder import load_pretrained_model
from llava.mm_utils import get_model_name_from_path
from llava.eval.run_llava import eval_model

# Load the LLaVA model paths
llava_model_path = "/content/drive/MyDrive/Colab Notebooks/Medical_Project/My_new_LLaVA/llava-ftmodel"
llava_med_model_path = "/content/drive/MyDrive/Colab Notebooks/Medical_Project/LLaVA_Med/fine_tuned-med-llava"

# Set up the Whisper pipeline for ASR
speech_to_text = pipeline("automatic-speech-recognition", model="openai/whisper-base")

# Args class to store arguments for LLaVA models
class Args:
    def __init__(self, model_path, model_base, model_name, query, image_path, conv_mode, image_file, sep, temperature, top_p, num_beams, max_new_tokens):
        self.model_path = model_path
        self.model_base = model_base
        self.model_name = model_name
        self.query = query
        self.image_path = image_path
        self.conv_mode = conv_mode
        self.image_file = image_file
        self.sep = sep
        self.temperature = temperature
        self.top_p = top_p
        self.num_beams = num_beams
        self.max_new_tokens = max_new_tokens

# Function to transcribe audio to text with error handling
def transcribe_audio(audio):
    try:
        transcription = speech_to_text(audio)["text"]
        return transcription
    except Exception as e:
        return f"Error in audio transcription: {str(e)}"

def predict_llava(image, question, temperature, max_tokens):
    try:
        temp_image_path = "temp_image.jpg"
        image.save(temp_image_path)

        # Ensure temperature is a float and max_tokens is an integer
        temperature = float(temperature) if temperature is not None else 0.7
        max_tokens = int(max_tokens) if max_tokens is not None else 50

        args = Args(
            model_path=llava_model_path,
            model_base=None,
            model_name=get_model_name_from_path(llava_model_path),
            query=question,
            image_path=temp_image_path,
            conv_mode=None,
            image_file=temp_image_path,
            sep=",",
            temperature=temperature,
            top_p=None,
            num_beams=1,
            max_new_tokens=max_tokens
        )

        # Call eval_model and return output
        output = eval_model(args)
    except Exception as e:
        output = f"Error in LLaVA prediction: {str(e)}"
    finally:
        if os.path.exists(temp_image_path):
            os.remove(temp_image_path)
    return output

# # Function to predict using LLaVA-Med with error handling
def predict_llava_med(image, question, temperature, max_tokens):
    temp_image_path = "temp_image_med.jpg"
    try:
        image.save(temp_image_path)
        command = [
            "python", "-m", "llava.eval.run_llava",
            "--model-name", llava_med_model_path,
            "--image-file", temp_image_path,
            "--query", question,
            "--temperature", str(temperature),
            "--max-new-tokens", str(max_tokens)
        ]
        result = subprocess.run(command, capture_output=True, text=True)
        output = result.stdout.strip() if result.returncode == 0 else f"Error in LLaVA-Med subprocess: {result.stderr.strip()}"
    except Exception as e:
        output = f"Error in LLaVA-Med prediction: {str(e)}"
    finally:
        if os.path.exists(temp_image_path):
            os.remove(temp_image_path)
    return output

# Main prediction function with handling for either text or audio input
def predict(model_name, image, text, temperature, max_tokens, audio=None):
    # Prioritize text input if both are provided, otherwise use audio transcription
    if text:
        question = text
    elif audio is not None:
        question = transcribe_audio(audio)
        print("Debug - Transcribed text from audio:", question)
    else:
        return "Error: Please provide either a text input or an audio input."

    try:
        # Model prediction based on selected model
        if model_name == "LLaVA":
            return predict_llava(image, question, temperature, max_tokens)
        elif model_name == "LLaVA-Med":
            return predict_llava_med(image, question, temperature, max_tokens)
        else:
            return "Error: Model name not recognized."
    except Exception as e:
        return f"Error in prediction function: {str(e)}"



/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:797: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/1.98k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/290M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/3.81k [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/283k [00:00<?, ?B/s]

...

normalizer.json:   0%|          | 0.00/52.7k [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/34.6k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/2.19k [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


preprocessor_config.json:   0%|          | 0.00/185k [00:00<?, ?B/s]

In [5]:
with gr.Blocks(css="""
    .gradio-container {
        padding-top: 0px;
        padding-bottom: 0px;
    }
    .input-image-box .gr-image-preview {
        width: 100%;  /* Full width */
        max-width: 700px; /* Adjust this value to control max width */
        height: 700px;  /* Adjust this value to control height */
        object-fit: contain;
    }
""") as demo:
    gr.Markdown("## Analyzing Neurological Patterns in Brain MRI Scans with AI Techniques")

    # Row for model selection
    with gr.Row():
        model_select = gr.Radio(choices=["LLaVA", "LLaVA-Med"], label="Select Model")

    # Row for image input, text input, and audio input
    with gr.Row():
        with gr.Column():
            image_input = gr.Image(type="pil", label="Input Image")
        with gr.Column():
            text_input = gr.Textbox(lines=2, placeholder="Type your question here...", label="Input Text")
            audio_input = gr.Audio(type="filepath", label="Upload or Record Speech")

    # Row for sliders
    with gr.Row():
        temperature_slider = gr.Slider(minimum=0.1, maximum=1.0, step=0.1, value=0.7, label="Temperature")
        max_tokens_slider = gr.Slider(minimum=1, maximum=256, step=1, value=50, label="Max Tokens")

    # Output textbox at the bottom
    output_text = gr.Textbox(lines=2, label="Output Text")

    # Submit button
    submit_button = gr.Button("Submit")
    submit_button.click(
        fn=predict,
        inputs=[model_select, image_input, text_input, temperature_slider, max_tokens_slider, audio_input],
        outputs=output_text
    )

# Launch the interface
demo.launch()


Running Gradio in a Colab notebook requires sharing enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://76539e863d59d4d9fd.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
